Check out [the conversation that wrote most of this](https://chat.openai.com/share/8b91ef50-cf4a-414c-a419-cbe10d31e2a0)

In [1]:
# Adjust this path as necessary to correctly reflect your project's structure
from pathlib import Path
PROJECT_ROOT = Path("..")
print(PROJECT_ROOT)

..


In [2]:
import re
from tqdm import tqdm
from typing import Tuple

# Define global variables that all templates may use
global_variables = ["crate_name", "crate_name_pascal"]

# Base directories
patterns_dir = Path("patterns")
matchers_dir = Path("matchers")

# Ensure the matchers directory exists
matchers_dir.mkdir(exist_ok=True)

# Function to find and process each template file
def process_templates(directory: Path):
    for template_file in tqdm(list(directory.glob("**/*.*")), desc="Processing templates"):
        process_template_file(template_file)

# Placeholder for the function to process each individual template file
def process_template_file(template_file: Path):
    print(f"Processing {template_file}")

# Start processing
process_templates(patterns_dir)

Processing templates: 100%|██████████| 16/16 [00:00<00:00, 21802.75it/s]

Processing patterns\new_plugin_with_types\Cargo.toml
Processing patterns\new_plugin_with_types\crates\plugins\Cargo.toml
Processing patterns\new_plugin_with_types\crates\plugins\src\lib.rs
Processing patterns\new_plugin_with_types\crates\{{crate_name}}\Cargo.toml
Processing patterns\new_plugin_with_types\crates\{{crate_name}}\src\lib.rs
Processing patterns\new_plugin_with_types\crates\{{crate_name}}\src\{{crate_name}}_types.rs
Processing patterns\new_plugin_with_types\crates\{{crate_name}}\src\{{crate_name}}_types_plugin.rs
Processing patterns\new_plugin_with_types\crates\{{crate_name}}_types\Cargo.toml
Processing patterns\new_plugin_with_types\crates\{{crate_name}}_types\src\lib.rs
Processing patterns\new_plugin_with_types\crates\{{crate_name}}_types\src\{{crate_name}}_types_plugin.rs
Processing patterns\new_tool\Cargo.toml
Processing patterns\new_tool\crates\plugins\Cargo.toml
Processing patterns\new_tool\crates\plugins\src\lib.rs
Processing patterns\new_tool\crates\{{crate_name}}_to

In [3]:
def template_content_matches(matcher_file: Path, template_content: str) -> bool:
    if not matcher_file.exists():
        return False
    with open(matcher_file, "r") as file:
        # Extract the commented-out template content from the matcher file
        existing_template_content = []
        for line in file:
            if line.startswith("# "):
                existing_template_content.append(line[2:].rstrip())
            else:
                # Stop reading once we reach the end of the commented-out template content
                break
        # Compare the joined existing template content with the input template content
        return "\n".join(existing_template_content) == template_content

In [4]:
def append_existing_content_and_path(destination_file: Path, py_so_far: str):
    file_content = ""
    if destination_file.exists():
        print(f"Appending existing content from {destination_file}")
        # We are about to clobber the old python splitting code
        # Give copilot context about what it used to look like
        with open(destination_file, "r") as existing_file:
            existing_content = existing_file.read()
        existing_lines = existing_content.splitlines()
        # trim the comments from the end of the file
        while existing_lines[-1].strip().startswith("#"):
            existing_lines.pop()
        # comment it out
        comment_lines = ["# " + line for line in existing_lines]
        
        file_content += "##### OLD CONTENT OF THIS FILE\n".join(comment_lines)
        file_content += "\n\n\n"
    else:
        print(f"Creating new file {destination_file}")

    # Calculate the relative path correctly
    relative_path = Path(*destination_file.parts[2:]).with_suffix("")
    workspace_path = PROJECT_ROOT / relative_path
    if workspace_path.exists():
        print(f"Appending existing content from {workspace_path}")
        file_content += "##### WORKSPACE CONTENT\n"
        file_content += "\n".join(["#"+x for x in (PROJECT_ROOT / relative_path).read_text(encoding="utf-8").split("\n")])
    else:
        print(f"No existing content found at {workspace_path}")
    
    # Append below
    return py_so_far + file_content

In [5]:
def write_matcher_file(destination_file: Path, template_content: str, variables: set):
    # Before overwriting, check if template content matches to preserve custom code
    if template_content_matches(destination_file, template_content):
        print(f"Template content for {destination_file} matches the existing file. Skipping overwrite.")
        return
    
    file_content = ""
    # Write the commented-out template content
    for line in template_content.splitlines():
        file_content += f"# {line}\n"
    file_content += "\nfrom typing import Tuple\n\n"
    
    # Write the stub function definition
    inner_type = ', '.join(['str'] * len(variables))
    file_content += f"def chunk(text: str) -> Tuple[{inner_type}]:\n"
    for var in variables:
        file_content += f"    # TODO: Implement logic for {var}\n"
    file_content += "    return ()\n\n"
    
    # Append existing content and path
    final_content = append_existing_content_and_path(destination_file, file_content)
    
    with open(destination_file, "w") as file:
        file.write(final_content)
    
    print(f"Updated matcher file for {destination_file}")

In [6]:
def extract_variables_from_template(template_content: str) -> set:
    # Regex to find all {{variable}} instances in the template
    variables = set(re.findall(r"\{\{([^}]+)\}\}", template_content))
    # Remove global variables
    return variables - set(global_variables)

def process_template_file(template_file: Path):
    # Read the content of the template file
    with open(template_file, "r") as file:
        template_content = file.read()
    
    # Extract unique variables from the template content
    variables = extract_variables_from_template(template_content)
    
    # Determine the destination folder and file for the matcher
    destination_folder = matchers_dir / template_file.relative_to(patterns_dir).parent
    destination_folder.mkdir(parents=True, exist_ok=True)
    destination_file = destination_folder / f"{template_file.name}.py"
    
    # Placeholder for writing to the matcher Python file
    print(f"Writing matcher for {template_file} to {destination_file}")
    write_matcher_file(destination_file, template_content, variables)

In [7]:
process_templates(patterns_dir)

Processing templates: 100%|██████████| 16/16 [00:00<00:00, 606.84it/s]

Writing matcher for patterns\new_plugin_with_types\Cargo.toml to matchers\new_plugin_with_types\Cargo.toml.py
Creating new file matchers\new_plugin_with_types\Cargo.toml.py
Appending existing content from ..\Cargo.toml
Updated matcher file for matchers\new_plugin_with_types\Cargo.toml.py
Writing matcher for patterns\new_plugin_with_types\crates\plugins\Cargo.toml to matchers\new_plugin_with_types\crates\plugins\Cargo.toml.py
Creating new file matchers\new_plugin_with_types\crates\plugins\Cargo.toml.py
Appending existing content from ..\crates\plugins\Cargo.toml
Updated matcher file for matchers\new_plugin_with_types\crates\plugins\Cargo.toml.py
Writing matcher for patterns\new_plugin_with_types\crates\plugins\src\lib.rs to matchers\new_plugin_with_types\crates\plugins\src\lib.rs.py
Creating new file matchers\new_plugin_with_types\crates\plugins\src\lib.rs.py
Appending existing content from ..\crates\plugins\src\lib.rs
Updated matcher file for matchers\new_plugin_with_types\crates\plugi